In [4]:
import math
import random
import time
import re
#from numba import njit
import pdb
from IPython.core.debugger import set_trace
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from lmfit import minimize, Parameters, Parameter, report_fit
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from math import sqrt
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from IPython.display import clear_output

#### Data

In [8]:
DF = pd.read_csv('data_plgm.csv')
DFF = DF
Time = DFF.Time
Season = DFF.Season

In [9]:

TOC = 1.1 # %
LakeArea = 582210000 # m2
# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

# DF0 = [['EpiP[g/m3]','HypP[g/m3]']]
# DFF= pd.DataFrame().assign(EpiP  =HED.E.values, HypP= HED.H.values)

'''Parameters Initialization'''
Sedv = DF.SedV[10]
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 


# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DFF[['EpiP']].values[0]#['EpiTP[g/m3]'][0]*DFF['EpiV'][0] 
Phyp[0] = DFF[['HypP']].values[0]#['HypTP[g/m3]'][0]*DFF['HypV'][0]

In [10]:
DFF

,Unnamed: 0,Time,EpiN[g/m3],HypN[g/m3],SedN[g/m3],EpiOx[g/m3],HypOx[g/m3],SedOx[g/m3],EpiP[g/m3],HypP[g/m3],...,Am,EpiZm,HypZm,EpiTemp,HypTemp,SedTemp,EntrP,LoadTP,EpiP,HypP
0,0,1981-01-01,0.652760,0.657200,0.639000,9.980000,7.193200,5450.000000,0.078220,0.105780,...,0.0,NaN,NaN,6.57100,6.57100,6.57100,0.000000e+00,1.453729e+08,2.877948e+12,6.229452e+12
1,1,1981-01-02,0.653314,0.658057,0.639762,9.952190,7.247010,5700.476190,0.078619,0.105368,...,0.0,NaN,NaN,6.57100,6.57095,6.56610,0.000000e+00,1.400055e+08,2.915730e+12,4.288388e+12
2,2,1981-01-03,0.654667,0.659034,0.640524,9.924295,7.300819,5950.952381,0.079021,0.104955,...,0.0,NaN,NaN,6.56820,6.57055,6.56325,0.000000e+00,1.577105e+08,2.928945e+12,4.290968e+12
3,3,1981-01-04,0.655993,0.659891,0.641286,9.894743,7.398371,6201.428571,0.079366,0.104314,...,0.0,NaN,NaN,6.55220,6.56980,6.56120,0.000000e+00,2.477765e+08,2.941628e+12,4.294095e+12
4,4,1981-01-05,0.657140,0.660629,0.642048,9.863790,7.540829,6451.904762,0.079684,0.104048,...,0.0,NaN,NaN,6.51825,6.56880,6.55785,0.000000e+00,5.481071e+08,2.988821e+12,4.138368e+12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13103,13103,2016-11-16,0.644667,0.530000,0.516667,9.746000,4.802667,4824.333333,0.006267,0.053667,...,404340000.0,14.0,212.0,12.22650,6.64340,6.57560,2.986882e+07,1.966168e+08,4.520086e+10,4.287331e+12
13104,13104,2016-11-17,0.648000,0.532000,0.520000,9.764000,4.949000,5047.000000,0.006400,0.054000,...,398160000.0,14.0,212.0,11.89400,6.64340,6.57565,0.000000e+00,1.953077e+08,4.616258e+10,4.293897e+12
13105,13105,2016-11-18,0.650667,0.534000,0.523333,9.784667,5.077333,5269.666667,0.006533,0.054333,...,404340000.0,14.0,212.0,11.91950,6.64340,6.57575,0.000000e+00,2.179022e+08,4.712430e+10,4.320159e+12
13106,13106,2016-11-19,0.654000,0.536000,0.526667,9.805333,5.236667,5477.333333,0.006467,0.054667,...,398160000.0,14.0,212.0,11.84500,6.64350,6.57580,0.000000e+00,2.563952e+08,4.664344e+10,4.326725e+12


#### PLGM model and functions

In [3]:
def PLGMplot(t,epip,epit,hypt,hypp):    
    fig, axs = plt.subplots(3, 1,figsize=(28,20))
    axs[0].plot(t,epit, label = 'True')
    axs[0].plot(t,epip, label = 'Obs')
    # axs[0].set_ylabel('Phosphorus [ug/L]')
    axs[0].set_ylabel('Phosphorus [g]')
    axs[0].set_xlabel('Year')
    axs[1].plot(t,hypt, label = 'True')
    axs[1].plot(t,hypp, label = 'Obs')
    axs[2].plot(t,sed, label = 'Obs')

In [3]:
def own_loss(y_true,y_pred,l1e,l2e,l1h,l2h):
    def sub_loss(l1x,l2x):
        # l1x = l1e    l2x = l2e
        rmse = ((y_true - y_pred) ** 2)
        r21 = ((y_true - np.mean(y_true)) ** 2)
        r22 = ((y_true - y_pred) ** 2)
        l  = np.array(rmse*l1x + (r21/r22)*l2x)[1:]
        return l
    le = sub_loss(l1e,l2e)
    lh = sub_loss(l1h,l2h)
    loss = le + lh
    return loss.ravel()

In [5]:
def own_opt_metric(params,DF):
    print(':)')
    le_opt  = params['le_opt']
    alpha  = params['alpha']
    beta  = params['beta']
    lh = 1 - le_opt
    l1e = le_opt*alpha
    l1h = lh*beta
    l2h = lh - l1h
    
    Output = 1; O = 0
    Pepi, Phyp = PLGM_OPT(params,DF)
    
    y_et = (DF['EpiP'])[:-2]
    y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

    y_et_pred = Pepi[:-2]
    y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

    y_ht = (DF['HypP'])[:-2]
    y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

    y_ht_pred = Phyp[:-2]
    y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]

    print('before filter nan')
    # Performance metric computation 
    y_e, y_e_pred,y_h,y_h_pred  = [list (x) for x in [y_e, y_e_pred,y_h,y_h_pred]]
    y_e, y_e_pred= [a for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))]
    y_h, y_h_pred= [a for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))]            
    
    print('before filter len')
    if len(y_e_pred)  < (y_et.shape[0])*0.85 or len(y_h_pred) < (y_et.shape[0])*0.85:
        loss = np.array([1e10]*len(y_et.shape[0]))
    else:
        loss_t = own_loss(y_true,y_pred,l1e,l2e,l1h,l2h)
    print(loss)
    clear_output(wait =True)
    return(loss.ravel())

In [4]:
def PLGM_OPT(params,DF):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        CburialP =params['CburialP']
        Csedi    = params['Csedi']
        CsedvE    = params['CsedvE']
        CsedvH    = params['CsedvH']
        le_opt = params['le_opt']
        alpha = params['alpha']
        beta = params['beta']

    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        CburialP =params0['CburialP']
        Csedi    = params0['Csedi']
        CsedvE    = params0['CsedvE']
        CsedvH    = params0['CsedvH']
        le_opt = params0['le_opt']
        alpha = params0['alpha']
        beta = params0['beta']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   
    Psed[0] = (SedimentArea * SedimentPConc)*PSED
    
    for i in Ndays[1:]:
        
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1] # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1] # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        # Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(HED['SedOx[ug/l]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))

        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            CPsedEpi = Csedi /2
            CPsedHyp = Csedi /2


        else:
            kmz = DF.Kz[i-1]#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            # CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            # CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365)) 
            CPsedEpi  = CsedvE
            CPsedHyp  = CsedvH

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]


    return(Pepi, Phyp)
        

    #return(df)

#### PLGM optimization

In [3]:

TOC = 1.1 # %
LakeArea = 582210000 # m2
# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

# DF0 = [['EpiP[g/m3]','HypP[g/m3]']]
# DFF= pd.DataFrame().assign(EpiP  =HED.E.values, HypP= HED.H.values)

'''Parameters Initialization'''
Sedv = DF.SedV[10]
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 


# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DFF[['EpiP']].values[0]#['EpiTP[g/m3]'][0]*DFF['EpiV'][0] 
Phyp[0] = DFF[['HypP']].values[0]#['HypTP[g/m3]'][0]*DFF['HypV'][0]

In [7]:
params = Parameters()

''' Sedimentation '''
# Phosphorus
params.add('ThetaSPepi',value = 1.25,  min=1, max=1.4)
params.add('ThetaSPhyp',value = 1.25,min=1, max=1.4)
params.add('TbaseSPepi',value = 11, min=1, max=15)
params.add('TbaseSPhyp',value = 11, min=1, max=15)
params.add('Fp', value = 0.45, min=0.01, max=0.95)
''' Resuspension '''
params.add('TbaseRp',value = 8,  min=1, max=15)
params.add('ThetaRp',value = 1.1,  min=1, max=1.2)
params.add('KsedP', value = 100*DF.SedV[10]*15*0.001/2, min=20*DF.SedV[10]*15*0.001 , max=100*DF.SedV[10]*15*0.001 )
params.add('RrecP', value = 7e-3,min= 14e-3 ,max =0 ) # only rate (d-1)
params.add('PSED',value = 500,min= 1 ,max = 1000 ) 
params.add('qE', value = 3,min= 0 ,max = 10)
''' Burial '''
params.add('CburialP', value = 1e-7, min = 1e-10, max = 1e-5)
params.add('le_opt', value = 0.25, min = 0.01, max = 0.5)
params.add('alpha', value = 0.25, min = 0.01, max = 0.5)
params.add('beta', value = 0.25, min = 0.01, max = 0.5)

# plt.plot(HED.iloc[:,-2])
LoadTP = DF['PLoad[g]']*1000
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

In [ ]:
''' Warning the optimization may take several hours to days''' 
#                    *Depend on pc specs*

In [112]:
result = minimize(own_opt_metric, params,kws={'DF': DF}, method='leastsq',nan_policy = 'propagate')
report_fit(result)

:)



KeyboardInterrupt



#### Optimized model

In [5]:

TOC = 1.1 # %
LakeArea = 582210000 # m2
# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

# DF0 = [['EpiP[g/m3]','HypP[g/m3]']]
# DFF= pd.DataFrame().assign(EpiP  =HED.E.values, HypP= HED.H.values)

'''Parameters Initialization'''
Sedv = DF.SedV[10]
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 


# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DFF[['EpiP']].values[0]#['EpiTP[g/m3]'][0]*DFF['EpiV'][0] 
Phyp[0] = DFF[['HypP']].values[0]#['HypTP[g/m3]'][0]*DFF['HypV'][0]

In [1]:
def PLGM_opt(params,DF,output = 'df', SA = False):
    
    '''
    __________________________________________________________________________________
    
    
                           Clean PLGM function to ouput predictions
    
    
    ___________________________________________________________________________________
    
    [ SA ]
                 ° True       : For sensitivity analysis
    
    [ output ]
                 °  'df'      :  Return output as DataFrame with all modeles variables
                 °   '*'      :  Return array with Pepi and Phyp
                 
                      °°(  '*' =  any input   )
    '''
       
    Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
    KsedP  = params0['KsedP']; SedDepth = 0.1
    TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
    RrecP = params0['RrecP'];  PSED = params0['PSED'];
    CburialP =params0['CburialP']
    Csedi    = params0['Csedi']
    CsedvE    = params0['CsedvE']
    CsedvH    = params0['CsedvH']
    le_opt = params0['le_opt']
    alpha = params0['alpha']
    beta = params0['beta']   

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   
    Psed[0] = (SedimentArea * SedimentPConc)*PSED
    
    for i in Ndays[1:]:
        
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1] # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1] # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        # Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(HED['SedOx[ug/l]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))

        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            CPsedEpi = Csedi /2
            CPsedHyp = Csedi /2


        else:
            kmz = DF.Kz[i-1]#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            # CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            # CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365)) 
            CPsedEpi  = CsedvE
            CPsedHyp  = CsedvH

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if output =='df':
        df = pd.DataFrame(data = {'EpiP[ug]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                          'EntrP[ug]':list(EntrP[:-2]),
                          'IepiP': list(IepiP[:-2]),
                          'SedPepi[ug]':list(SedPepi[:-2]),
                          'Ep[ug]':list(Ep[:-2]),
                          'Hvd':list(Hvd[:-2]), 
                          'HypP[ug]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                          'SedPhyp[ug]':list(SedPhyp[:-2]),
                          'Rp[ug]':list(Rp[:-2]),
                          'IsedP[ug]':list(IsedP[:-2]),
                          'Psed[ug]':list(Psed[:-2]),
                          'Bp[ug]':list(Bp[:-2])

                         })
        return(df)
    else:
        return(Pepi[:-2], Phyp[:-2])
        

    #return(df)

In [2]:
# plt.plot(HED.iloc[:,-2])
LoadTP = DF['PLoad[g]']*1000
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi




params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,           
 'PSED': 2,
 'CburialP': 0.01,
 'CsedvH' : 1e-20,
 'CsedvE' : 1e-20,
 'Csedi'  : 1e-15,
 'KsedP': (1.846781e+13)/5,
 'RrecP': 1e-4,#1e-05,
 'le_opt': 0.3 ,
 'alpha':  0.6,
 'beta': 0.3 }

Output =1 ;O = 'df'
k=PLGM_opt(params0,DF)
epit = DFF[['EpiP']][:-2].values.T[0]*1e-6#/DF.iloc[0:13106,:]['EpiV'].values
epip = k.iloc[:,0].values*1e-6#/DF.iloc[0:13106,:]['EpiV'].values
t     = pd.to_datetime(DFF.loc[:,'Time'][:-2].values)#.T[0]
hypp = k.iloc[:,6]*1e-6#/DF.iloc[0:13106,:]['HypV']
hypt =DFF[['HypP']][:-2].values.T[0]*1e-6#/DF.iloc[0:13106,:]['HypV']

sed = k.iloc[:,-2]*1e-6#/Sedv
PLGMplot(t,epip,epit,hypt,hypp)

NameError: name 'DF' is not defined

In [100]:
''' Exporting PLGM Output'''

k['PEPItrue[ug]'] = DFF.iloc[:13106,0].values
k['HYPtrue[ug]']  = DFF.iloc[:13106,1].values
k['EpiV[L]']     = DF.iloc[0:13106,:]['EpiV'].values
k['HypV[L]']     = DF.iloc[0:13106,:]['HypV'].values
k['Time']     = DF.Time.values[:13106]
k['Sedv[L]']     = Sedv
k['Season']      = DF.Season[:13106].values
# k.to_csv('plgm.csv')

#### Sensivity

In [125]:
''' '''
sen_n = ['kmzvi','PSEDvi','Cburialvi','EntrPvi','Lvi','Oxvi','svi','Tvi','Evi','Res']
sen_v = [[0,1.5,2,5,10],   # Vertical eddy diffusion constant
         [0,1.5,2,5,10],   # Initial Sediment P concentration
         [0,1.5,2,5,10],   # Burial rate
         [0,1.5,2,5,10],   # Etrainment
         [0,1.5,2,5,10],   # Pload
         [0,1.5,2,5,10],   # Oxygen
         [0,1.5,2,5,10],   # Sedimentation rate
         [0,1.5,2,5,10],   # Temperature
         [0,1.5,2,5,10],   # Export
         [0,1.5,2,5,10]]   # Release
for ix, i in enumerate(sen_n):
    print(i[:-1])
    exec(f'{i} = {sen_v[ix]}')

kmzv
PSEDv
Cburialv
EntrPv
Lv
Oxv
sv
Tv
Ev
Re


In [128]:
def PLGM(params,DF):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        CburialP =params0['CburialP']
        Csedi    = params0['Csedi']
        CsedvE    = params0['CsedvE']
        CsedvH    = params0['CsedvH']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp*Tv - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp*Tv - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp*Tv - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   
    Psed[0] = (SedimentArea * SedimentPConc)*PSED*PSEDv
    
    for i in Ndays[1:]:
        
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1]*EntrPv # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1]*EntrPv # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        # Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(Oxv*HED['SedOx[ug/l]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))

        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            CPsedEpi = Csedi /2
            CPsedHyp = Csedi /2

        else:
            kmz = DF.Kz[i-1]*kmzv#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            # CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            # CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365)) 
            CPsedEpi  = CsedvE
            CPsedHyp  = CsedvH

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv*sv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv*sv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]*Lv    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]*Lv        # To sediment
        '''Burial'''
        Bp[i] = Cburialv*CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        DF0 = DF.copy()
        loss =own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
        return(loss)
        #
    elif Output == 0 and not cond:
        loss = np.nan
        return(loss)
    else:
        if O =='df':
            DF0 = DF.copy()
            df = pd.DataFrame(data = {
                              'Time': Time[:13106],
                               'Season':  Season[:13106],
                              'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            if cond:
                DF0 = DF.copy()
                loss = own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
            else:
                loss = np.nan
                
            return(df,loss)
        else:
            return(Pepi[:-2], Phyp[:-2])

In [129]:
params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
           
 'PSED': 2,
 'CburialP': 0.01,
 'CsedvH' : 1e-20,
 'CsedvE' : 1e-20,
 'Csedi'  : 1e-15,
 'KsedP': (1.846781e+13)/5,
 'RrecP': 1e-5,#1e-05,
 'le_opt': 0.3 ,
  'alpha':  0.6,
  'beta': 0.3 }

output = 1; Data = 1; O = 'df'
DF0 =DF.copy()
for i in sen_n:
    exec(f'{i[:-1]} = 1')
pred = PLGM(params0,DF0)[0].set_index('Time')
score = PLGM(params0,DF0)[1]

overall_baseline = score 
overall_importances = {}
f = lambda l,v: [x for x in l if x != v]

for ix,i in enumerate(sen_n):
    for vx,v in enumerate(sen_v[ix]): 
        clear_output(wait = True)
        exec(f'{i[:-1]} = {v}')
        score = 0
        for rx,r in enumerate(f(sen_n,i)):
            exec(f'{r[:-1]} = {1}')            
        DF0 = DF.copy()
        Output = 0; Data = 1
        score +=  PLGM(params0,DF0)
        print(i , score, overall_baseline)
        
        if i not in overall_importances:             
            overall_importances[i] = {vx:np.abs(overall_baseline - score)}
        else:
            overall_importances[i][vx] = np.abs(overall_baseline - score)
        

TypeError: own_metric() got multiple values for argument 'Pepi'

In [133]:
fi = pd.DataFrame(overall_importances).apply(lambda x: np.nanmax(x)).to_dict()
v = np.array(list(fi.values()))/sum(np.array(list(fi.values())))
sorted_indices = np.argsort(v)[::-1]
names = list(fi.keys())
nm = {}
for i in range(len(sorted_indices)):
    nm[names[sorted_indices[i]]] = {'value':fi[names[sorted_indices[i]]]}
    # print(f"{i+1}. {names[sorted_indices[i]]},{fi[names[sorted_indices[i]]]}")
FI = pd.DataFrame(nm).T
FI['value'] = np.round(FI['value'] / np.sum(FI['value']),3)
FI

,value
EntrPvi,0.398
Evi,0.361
Lvi,0.180
kmzvi,0.056
svi,0.003
Tvi,0.001
PSEDvi,0.001
Oxvi,0.000
Cburialvi,0.000


#### Adjusting

In [40]:

                                    ''' Mechanistic Model for the Mass Balance '''
#r2_e,r2_h,rmse_e,rmse_h,loss = [None]*5
# ODE
# To monitorize
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

def PLGM(params,DF):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   
    Psed[0] = (SedimentArea * SedimentPConc)*PSED
    
    for i in Ndays[1:]:
        
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1] # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1] # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.Kz[i-1]#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        clear_output()
        y_et = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
        y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

        y_et_pred = Pepi[:-2]
        y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

        y_ht = (DF['HypTP[g/m3]']*DF.HypV)[:-2]
        y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

        y_ht_pred = Phyp[:-2]
        y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


        r2_e    = r2_score(y_e,y_e_pred)
        rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

        r2_h    = r2_score(y_h,y_h_pred)
        rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))
        
        # 
        # l2 = 1 - l1
        # l1e = l2*0.3 
        # l2h = l2-l1e
        loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
        #loss = rmse_e + rmse_h + abs(1-r2_e) + abs(1-r2_h)

        print('-----Results-----')
        print(f'R2=>  Epi: {r2_e:.2%}  Hyp: {r2_h:.2%}')
        print(f'rmse=>  Epi: {rmse_e:.4}  Hyp: {rmse_h:.4}')
        print(f'loss: {loss:.4}\n')
        return(loss)
        #
    elif Output == 0 and not cond:
        loss = random.uniform(10, 15)
        return(loss)
    else:
        if O =='df':
            df = pd.DataFrame(data = {'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            return(df)
        else:
            return(Pepi[:-2], Phyp[:-2])

KeyError: 'EpiTP[g/m3]'

In [ ]:

                                    ''' Mechanistic Model for the Mass Balance '''

LoadTP = DF['PLoad[g]']
LoadTN = DF['NLoad[g]']
a_ratio  = 0.5   # Ratio C:Chla content' 
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi
#r2_e,r2_h,rmse_e,rmse_h,loss = [None]*5
Sedv = DF.SedV[10]
# ODE
# To monitorize
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 1000,
 'CburialP': 0.01,
 'l1': 0.3
}
Output = 1;O = 'df'
k = PLGM(params0)

In [ ]:
Output = 0; Data = 0
def error(params,DF):
    y_pred = PLGM(params,DF)#.values#.iloc[:,0].values
    #y_true = (DF['EpiTP[g/m3]'][:13106]*DF.EpiV[:13106]).values
    y_true = DF1[['EpiTP[g/m3]','HypTP[g/m3]']][:13106].values.T
    return (y_true-y_pred).ravel()
result = minimize(error, params,kws={'DF': DF}, method='leastsq',nan_policy = 'propagate' )
report_fit(result)

#### PLOTs

In [ ]:
##### Liupa

Output = 0; Data = 1
fig, axs = plt.subplots(3, 2,figsize=(28,12))
axs = axs.ravel()
axs[2].plot(DF.Time[:-2],PLGM(params,DF)[0])
axs[2].set_title('Epi Predicted', fontsize = 15, fontweight='bold')
axs[2].set_ylabel('Total Phosphorus [g]', fontsize = 15);

axs[0].set_title('Epi Observed', fontsize = 15, fontweight='bold')
axs[0].plot(DF.Time,DF['EpiTP[g/m3]']*DF.EpiV)
axs[0].set_ylabel('Total Phosphorus [g]', fontsize = 15);


axs[3].plot(DF.Time[:-2],PLGM(params,DF)[1])
axs[3].set_title('Hyp Predicted', fontsize = 15, fontweight='bold')


axs[1].set_title('Hyp Observed', fontsize = 15, fontweight='bold')
axs[1].plot(DF.Time,DF['HypTP[g/m3]']*DF.HypV);

Output = 1; Data = 1
axs[4].plot(DF.Time[:-2],PLGM(params,DF)['Psed[g]'])
axs[4].set_title('Sediment Phosphorus', fontsize = 15, fontweight='bold')
axs[4].set_ylabel('Total Phosphorus [g]', fontsize = 15);

In [ ]:
#k = 1
k=0
n = 365*k

m = n+365

Output = 0; Data =1

fig, axs = plt.subplots(1, 3,figsize=(30,8))
axs = axs.ravel()
axs[0].plot(DF.Time[n:m],(DF['EpiTP[g/m3]']*DF.EpiV)[n:m], label = 'y_true')
axs[0].plot(DF.Time[n:m],PLGM(params,DF)[0][n:m],'-' ,label = 'y_predicted')
axs[0].set_title('Epilimnion')
r2=r2_score((DF['EpiTP[g/m3]']*DF.EpiV)[n:m],PLGM(params,DF)[0][n:m])
axs[0].text(DF.Time[n:m][int((n+m)/2)],1e11,f" R2: {r2:.2%}", ha="center", va="center", size=20)


axs[1].plot(DF.Time[n:m],(DF['HypTP[g/m3]']*DF.HypV)[n:m],label ='y_true')
axs[1].plot(DF.Time[n:m],PLGM(params,DF)[1][n:m],label ='y_predicted')
axs[1].set_title('Hypolimnion')
r2=r2_score((DF['HypTP[g/m3]']*DF.HypV)[n:m],PLGM(params,DF)[1][n:m])
axs[1].text(DF.Time[n:m][int((n+m)/2)],PLGM(params,DF)[1][n+30],f" R2: {r2:.2%}", ha="center", va="center", size=20)


Output = 1; Data =1
#axs[2].plot(DF.Time[n:m],(DF['SedTP[g/m3]']*DF.SedV)[n:m],label ='y_true')
axs[2].plot(DF.Time[n:m],PLGM(params,DF)['Psed[g]'][n:m],label ='y_predicted')
axs[2].set_title('Sediment')


axs[0].set_ylabel('Total Phosphorus [g]')

axs[0].legend()
axs[1].legend()
axs[2].legend()
k+=1

In [ ]:
PLGM(params,DF).iloc[1:,:]
DF['HypP[g/m3]']*DF.EpiV

In [ ]:
Output = 1; Data =1
DF_pred = PLGM(params,DF).iloc[1:,:]
DF_pred = DF_pred.assign(Time = list(DF.Time)[1:-2])
p = 'C:\\Users\gabri\OneDrive\Documentos\GitHub\Thesis\Python\DF_pred.csv'
DF_pred.to_csv(p, sep = ',',encoding = 'utf-8')

In [ ]:
Output = 0; Data =1
y = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]#[n:m]#(DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
y_pred = PLGM(params,DF)[0]#[n:m]#PLGM(params,DF)[0][:]
r2 = r2_score(y,y_pred)
mae = mean_absolute_error(y,y_pred)
mse = mean_squared_error(y,y_pred)
rss = np.sum((y - y_pred) ** 2)
mape = np.mean(np.abs((y - y_pred) / y)) * 100
msle = mean_squared_error(np.log(y + 1), np.log(y_pred + 1))
cod = 1 - mse / np.var(y)
msd = np.mean(y_pred - y)
mda = np.mean(np.sign(y_pred - y) == np.sign(y - np.mean(y)))
rmsle = np.sqrt(mean_squared_error(np.log(y + 1), np.log(y_pred + 1)))


In [ ]:
# adjusted R-squared
n = len(y)#len(y_true) # sample size
p = 25#len(features) # number of independent variables
adj_r2 = 1 - ((1 - r2) * (n - 1)) / (n - p - 1)
# Akaike information criterion
nll = -(n / 2) * (1 + np.log(2 * np.pi * mse))
aic = 2 * p - 2 * nll
# Bayesian information criterion
bic = np.log(n) * p - 2 * nll

In [ ]:
print(f'\n\
 R2:  {r2:.2%}\n\
 MAE: {mae:.1e}\n\
 MSE: {mse:.1e}\n\
 RMSE: {(np.sqrt(mse)):.1e}\n\
 RSS: {rss:.1e}\n\
 MAPE: {mape:.1e}\n\
 MSLE: {msle:.1e}\n\
 COD: {cod:.1e}\n\
 MSD: {msd:.1e}\n\
 MDA:{mda:.1e}\n\
 RMSLE: {rmsle:.1e}\n\
 BIC: {bic:.1e}\n\
 AIC: {aic:.1e}\n\
 adj_r2: {adj_r2:.2%}')

#### Sensitive analisis

In [5]:
def own_metric(DF0,Pepi = None,Phyp = None,l1 = 0.3,t = 1, s= None):
    if t == 1:
        clear_output()
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        y_et = (DF0['EpiTP[g/m3]']*DF0.EpiV)[:-2]
        y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

        y_et_pred = Pepi[:-2]
        y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

        y_ht = (DF0['HypTP[g/m3]']*DF0.HypV)[:-2]
        y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

        y_ht_pred = Phyp[:-2]
        y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


        r2_e    = r2_score(y_e,y_e_pred)
        rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

        r2_h    = r2_score(y_h,y_h_pred)
        rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

        loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
    else:
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        DF0.set_index('Time',inplace = True)
        DF0['EpiTP[g]'] = DF0['EpiTP[g/m3]' ]*DF0.EpiV
        DF0['HypTP[g]'] = DF0['HypTP[g/m3]' ]*DF0.HypV
        mask = DF0.Season == s
        te = DF0[mask].groupby([DF0[mask].index.year,DF0[mask].index.month])[['EpiTP[g]']]\
                         .agg(np.median).values
        th = DF0[mask].groupby([DF0[mask].index.year,DF0[mask].index.month])[['HypTP[g]']]\
                         .agg(np.median).values
        y_et = te[:-2]
        y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

        y_et_pred = Pepi[:-2]
        y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

        y_ht = th[:-2]
        y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

        y_ht_pred = Phyp[:-2]
        y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


        r2_e    = r2_score(y_e,y_e_pred)
        rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

        r2_h    = r2_score(y_h,y_h_pred)
        rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))
        # DF.reset_index(inplace = True)
        loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
    return(loss)

##### FI

In [28]:
params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'nP': 16645161.290322721,
 'q': 1.2874561841905887,
 'PSED': 1,
 'CburialP': 0.2,
 'CPsedEpi': 0.00010563582513371828,
 'CPsedHyp': 0.0035651695362787727,
 'l1': 0.3}
def PLGM(params):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        l1 = params0['l1'] 
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp*Tv - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp*Tv - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp*Tv - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   

    Psed[0] = (SedimentArea * SedimentPConc)*PSED*PSEDv
    
    for i in Ndays[1:]:
        # LoadTP = DF['PLoad[g]']*Lv
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1]*EntrPv # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1]*EntrPv # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(Oxv*DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.Kz[i-1]*kmzv#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]*Lv    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]*Lv        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth*Cburialv    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        DF0 = DF.copy()
        loss = own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
        return(loss)
        #
    elif Output == 0 and not cond:
        loss = np.nan
        return(loss)
    else:
        if O =='df':
            DF0 = DF.copy()
            df = pd.DataFrame(data = {
                              'Time': Time[:13104],
                               'Season':  Season[:13104],
                              'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            if cond:
                DF0 = DF.copy()
                loss = own_metric(DF,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
            else:
                loss = np.nan
            return(df,loss)
        else:
            return(Pepi[:-2], Phyp[:-2])
        

    #return(df)

In [35]:
DFF = pd.read_csv('DF.csv')
DF = DF.assign(SedTemp = DFF.SedTemp, 
               EpiTemp =DFF.EpiTemp, 
               HypTemp = DFF.HypTemp,
               Strat = DFF.Strat,
               EpiV  = DF['EpiV[L]'],
               HypV = DF['HypV[L]'],
               Kz   = DFF.Kz,
               HypZm = DFF.HypZm, 
               EpiZm = DFF.EpiZm,
               Time  = DFF.Time) 
DF['SedOx[g/m3]'] = DFF['SedOx[g/m3]']*1000
DF['SedV'] = DFF['SedV']

params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'nP': 16645161.290322721,
 'q': 1.2874561841905887,
 'PSED': 1,
 'CburialP': 0.2,
 'CPsedEpi': 0.00010563582513371828,
 'CPsedHyp': 0.0035651695362787727,
 'l1': 0.3}
# params0 = {
#  'ThetaSPepi': 1.3123834064195374,
#  'ThetaSPhyp': 1.1241227447142852,
#  'TbaseSPepi': 4.113509201569038,
#  'TbaseSPhyp': 14.179827943922088,
#  'Fp': 0.24687844315278318,
#  'TbaseRp': 7.902335061473191,
#  'ThetaRp': 1.1264026422029476,
#  'KsedP': 1613746294.5171556,
#  'RrecP': 1e-05,
#  'PSED': 500,
#  'CburialP': 0.01,
#  'l1': 0.3
# }
Time = DF.Time
Season = DF.Season

In [36]:
sen_n = ['kmzvi','PSEDvi','Cburialvi','EntrPvi','Lvi','Oxvi','Sedvi','Tvi','Evi']
sen_v = [[0,1.5,2,5,10],   # Kmz
         [0,1.5,2,5,10],   # Psed
         [0,1.5,2,5,10],   # Cburialv
         [0,1.5,2,5,10],   # EntrPv
         [0,1.5,2,5,10],   # Pload
         [0,1.5,2,5,10],   # Oxv
         [0,1.5,2,5,10],   # Sedv
         [0,1.5,2,5,10],   # Tv
         [0,1.5,2,5,10]]   # Ev
for ix, i in enumerate(sen_n):
    exec(f'{i} = {sen_v[ix]}')

In [37]:
for i in sen_n:
    exec(f'{i[:-1]} = 1')
Output = 1; Data = 1; O = 'df'
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

C:\Users\gabri\AppData\Local\Temp\ipykernel_920\1001659811.py:97: RuntimeWarning: overflow encountered in scalar multiply
  Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
C:\Users\gabri\AppData\Local\Temp\ipykernel_920\1001659811.py:119: RuntimeWarning: invalid value encountered in scalar add
  Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]


In [ ]:
Output = 1; Data = 1; O = 'df'
for i in sen_n:
    exec(f'{i[:-1]} = 1')
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

overall_baseline = score 
seasonal_baseline = []

overall_importances = {}
seasonal_importances = {}

for ix, i in enumerate(pred.Season.unique()):
    
    pe = pred[pred.Season == i]\
                .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['EpiTP[g]']]\
                .agg(np.median).values.T[0]

    ph = pred[pred.Season == i]\
                .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['HypTP[g]']]\
                .agg(np.median).values.T[0]
    DF0 = DF.copy()
    seasonal_baseline.append(own_metric(DF0,Pepi = pe,Phyp = ph,l1= 0.3, t = 0,s= i))
    
f = lambda l,v: [x for x in l if x != v]

for ix,i in enumerate(sen_n):
    for vx,v in enumerate(sen_v[ix]): 
        clear_output(wait = True)
        exec(f'{i[:-1]} = {v}')
        score = 0
        for rx,r in enumerate(f(sen_n,i)):
            exec(f'{r[:-1]} = {1}')            
        DF0 = DF.copy()
        Output = 0; Data = 1
        score +=  PLGM(params0)
        print(i , score, overall_baseline)
        
        if i not in overall_importances:             
            overall_importances[i] = {vx:np.abs(overall_baseline - score)}
        else:
            overall_importances[i][vx] = np.abs(overall_baseline - score)
        
        Output = 1; Data = 1; O = 'df'           
        pred = PLGM(params0)[0].set_index('Time')

#####  Free Parameters importances

In [ ]:

                                    ''' Mechanistic Model for the Mass Balance '''
#r2_e,r2_h,rmse_e,rmse_h,loss = [None]*5
Sedv = DF.SedV[10]
# ODE
# To monitorize
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

params = [Real(1,1.4,name='ThetaSPepi'),                
                Real(1,1.4, name='ThetaSPhyp'),
                Real(1,15,name='TbaseSPepi'),
                Real(1,15,name='TbaseSPhyp'), 
                Real(0.1,0.9,name='Fp'),
                Real(1,15,name='TbaseRp'),
                Real(1,1.2,name='ThetaRp'),
                #Real(20*30.973761*0.001,100*30.973761*0.001,name='KsedP'),
                Real(20*DF.SedV[10]*15*0.001,100*DF.SedV[10]*15*0.001,name='KsedP'),
                #Real(-30*DF.SedV[10]*30.973761*0.001,30*DF.SedV[10]*30.973761*0.001,name='RrecP'),
                Real(1e-6,1e-2,name='RrecP'),
                #Real(0,10*10*DF.SedV[10]*1e-3,name ='nP'), # 5* is my invention
                #Real(0,20,name ='q'),
                Real(1,10e20,name='PSED'),
                Real(1e-3,0.4,name='CburialP'),
                #Real(1e-6,2,name='CPsedEpi'),
                #Real(1e-5,1,name='CPsedEpi'),
                #Real(1e-5,1,name='CPsedHyp'),
                #Real(1e-20,1e20,name='ed'),
                #Real(1e-20,1e20,name='hd'),
                #Real(0.1,1,name='cfactor'),
                #Real(1e-6,1e-2,name='Csed'),
                Real(0.1,0.3,name = 'l1')]

# Define the objective function to be minimized
@use_named_args(params)
def PLGM(**params):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   
    Psed[0] = (SedimentArea * SedimentPConc)*PSED
    
    for i in Ndays[1:]:
        
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1] # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1] # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.Kz[i-1]#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
      
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0:
        DF0 = DF.copy()
        loss = own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
#         clear_output()
#         y_et = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
#         y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

#         y_et_pred = Pepi[:-2]
#         y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

#         y_ht = (DF['HypTP[g/m3]']*DF.HypV)[:-2]
#         y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

#         y_ht_pred = Phyp[:-2]
#         y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


#         r2_e    = r2_score(y_e,y_e_pred)
#         rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

#         r2_h    = r2_score(y_h,y_h_pred)
#         rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

#         loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)


        return(loss)
        #
    else:
        if O =='df':
            DF0 = DF.copy()
            df = pd.DataFrame(data = {
                              'Time': Time[:13106],
                               'Season':  Season[:13106],
                              'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })

            DF0 = DF.copy()
            loss = own_metric(DF,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
            return(df,loss)
        else:
            return(Pepi[:-2], Phyp[:-2])
        

    #return(df)

In [ ]:
def own_metric(DF0,Pepi = None,Phyp = None,l1 = 0.3,t = 1, s= None):
    if t == 1:
        clear_output()
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        y_et = (DF0['EpiTP[g/m3]']*DF0.EpiV)[:-2]
        y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

        y_et_pred = Pepi[:-2]
        y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

        y_ht = (DF0['HypTP[g/m3]']*DF0.HypV)[:-2]
        y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

        y_ht_pred = Phyp[:-2]
        y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]

     
            

        y_e, y_e_pred,y_h,y_h_pred  = [list (x) for x in [y_e, y_e_pred,y_h,y_h_pred]]
        y_e, y_e_pred= [a for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))]
        y_h, y_h_pred= [a for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))]

            
        r2_e    = r2_score(y_e,y_e_pred)
        rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

        r2_h    = r2_score(y_h,y_h_pred)
        rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

        loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
    else:
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        DF0.set_index('Time',inplace = True)
        DF0['EpiTP[g]'] = DF0['EpiTP[g/m3]' ]*DF0.EpiV
        DF0['HypTP[g]'] = DF0['HypTP[g/m3]' ]*DF0.HypV
        mask = DF0.Season == s
        te = DF0[mask].groupby([DF0[mask].index.year,DF0[mask].index.month])[['EpiTP[g]']]\
                         .agg(np.median).values
        th = DF0[mask].groupby([DF0[mask].index.year,DF0[mask].index.month])[['HypTP[g]']]\
                         .agg(np.median).values
        y_et = te[:-2]
        y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

        y_et_pred = Pepi[:-2]
        y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

        y_ht = th[:-2]
        y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

        y_ht_pred = Phyp[:-2]
        y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]
        
        

        y_e, y_e_pred,y_h,y_h_pred  = [list (x) for x in [y_e, y_e_pred,y_h,y_h_pred]]
        y_e, y_e_pred= [a for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_e, y_e_pred) if not (np.isnan(a) or np.isnan(b))]
        y_h, y_h_pred= [a for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))], [b for a, b in zip(y_h, y_h_pred) if not (np.isnan(a) or np.isnan(b))]
        
        
        r2_e    = r2_score(y_e,y_e_pred)
        rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

        r2_h    = r2_score(y_h,y_h_pred)
        rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))
        # DF.reset_index(inplace = True)
        loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
    return(loss)

In [ ]:
LoadTP = DF['PLoad[g]']
LoadTN = DF['NLoad[g]']
a_ratio  = 0.5   # Ratio C:Chla content' 
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

In [ ]:
Time = DF.Time
Season = DF.Season
DF0 = DF.copy()               
''' Mechanistic Model for the Mass Balance '''
# To monitorize
Ndays = range(0,len(DF0.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF0.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF0.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF0['EpiTP[g/m3]'][0]*DF0['EpiV'][0] 
Phyp[0] = DF0['HypTP[g/m3]'][0]*DF0['HypV'][0]

params00 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 800,
 'CburialP': 0.01,
 'l1': 0.3
}

params0 = params00.copy()
Output = 1; Data = 1; O = 'df'

pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

overall_baseline = score 
seasonal_baseline = []

overall_importances = {}
seasonal_importances = {}

# for ix, i in enumerate(pred.Season.unique()):
    
#     pe = pred[pred.Season == i]\
#                 .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['EpiTP[g]']]\
#                 .agg(np.median).values.T[0]

#     ph = pred[pred.Season == i]\
#                 .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['HypTP[g]']]\
#                 .agg(np.median).values.T[0]
#     DF0 = DF.copy()
#     seasonal_baseline.append(own_metric(DF0,Pepi = pe,Phyp = ph,l1= 0.3, t = 0,s= i))
    
# f = lambda l,v: [x for x in l if x != v]

# for vx,v in params00.items():
#     params0 = params00.copy()
#     clear_output(wait = True)
#     params0[vx] == 0
#     score = 0            
#     DF0 = DF.copy()
#     Output = 0; Data = 1
#     score +=  PLGM(params0)
#     print(i , score, overall_baseline)
#     overall_importances[vx] = {np.abs(overall_baseline - score)} 

#     Output = 1; Data = 1; O = 'df'           
#     pred = PLGM(params0)[0].set_index('Time')
for sx, s in enumerate(pred.Season.unique()):
    print('Seaon sensitive')
    score = 0
    pe = pred[pred.Season == s]\
                .groupby([pred[pred.Season == s].index.year,pred[pred.Season == s].index.month])[['EpiTP[g]']]\
                .agg(np.median).values.T[0]

    ph = pred[pred.Season == s]\
                .groupby([pred[pred.Season == s].index.year,pred[pred.Season == s].index.month])[['HypTP[g]']]\
                .agg(np.median).values.T[0]
    DF0 = DF.copy()
    # if np.isnan(np.asarray(pe[:-2])).any():
    #     score = np.nan
    # else:
    score = own_metric(DF0,Pepi = pe,Phyp = ph,l1= 0.3, t = 0,s= s)

    print(vx, s, seasonal_baseline[sx],score)
    if s not in seasonal_importances:
        seasonal_importances[s]={vx:np.abs(seasonal_baseline[sx] - score)}
    else:
        seasonal_importances[s][vx] = np.abs(seasonal_baseline[sx] - score)

    

           

In [ ]:
overall_importances

##### Feature importances

In [15]:

#                                     ''' Mechanistic Model for the Mass Balance '''
# #r2_e,r2_h,rmse_e,rmse_h,loss = [None]*5
# # ODE
# # To monitorize
# Time = DF.Time
# Season = DF.Season
# DF0 = DF.copy()
# Ndays = range(0,len(DF.index),1)
# V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

# for v in V:
#     exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

# #
# '''Parameters Initialization'''

# # Calculate epi and hypo volume vectors, which should always add up to lake volume
# HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
# EpiV = np.asarray(DF.EpiV)

# # Change in epilimnetic volume (m3), used for entrainment
# # where a +change means transfer of volume from hypo to epi
# dEpiV = np.insert(np.diff(EpiV),0,0)

# Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
# Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]
DFF = pd.read_csv('DF.csv')
DF = DF.assign(SedTemp = DFF.SedTemp, 
               EpiTemp =DFF.EpiTemp, 
               HypTemp = DFF.HypTemp,
               Strat = DFF.Strat,
               EpiV  = DF['EpiV[L]'],
               HypV = DF['HypV[L]'],
               Kz   = DFF.Kz,
               HypZm = DFF.HypZm, 
               EpiZm = DFF.EpiZm,
               Time  = DFF.Time) 
DF['SedOx[g/m3]'] = DFF['SedOx[g/m3]']*1000
DF['SedV'] = DFF['SedV']

# Time = DF.Time
# Season = DF.Season
# DF0 = DF.copy()
# Ndays = range(0,len(DF.index),1)
# V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

# for v in V:
#     exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

# #
# '''Parameters Initialization'''

# # Calculate epi and hypo volume vectors, which should always add up to lake volume
# HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
# EpiV = np.asarray(DF.EpiV)

# # Change in epilimnetic volume (m3), used for entrainment
# # where a +change means transfer of volume from hypo to epi
# dEpiV = np.insert(np.diff(EpiV),0,0)

# Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
# Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

# params = [Real(1,1.4,name='ThetaSPepi'),                
#                 Real(1,1.4, name='ThetaSPhyp'),
#                 Real(1,15,name='TbaseSPepi'),
#                 Real(1,15,name='TbaseSPhyp'), 
#                 Real(0.1,0.9,name='Fp'),
#                 Real(1,15,name='TbaseRp'),
#                 Real(1,1.2,name='ThetaRp'),
#                 #Real(20*30.973761*0.001,100*30.973761*0.001,name='KsedP'),
#                 Real(20*DF['Sedv[L]'][10]*15*0.001,100*DF['Sedv[L]'][10]*15*0.001,name='KsedP'),
#                 #Real(-30*DF.SedV[10]*30.973761*0.001,30*DF.SedV[10]*30.973761*0.001,name='RrecP'),
#                 Real(1e-6,1e-2,name='RrecP'),
#                 #Real(0,10*10*DF.SedV[10]*1e-3,name ='nP'), # 5* is my invention
#                 #Real(0,20,name ='q'),
#                 Real(1,10e20,name='PSED'),
#                 Real(1e-3,0.4,name='CburialP'),
#                 #Real(1e-6,2,name='CPsedEpi'),
#                 #Real(1e-5,1,name='CPsedEpi'),
#                 #Real(1e-5,1,name='CPsedHyp'),
#                 #Real(1e-20,1e20,name='ed'),
#                 #Real(1e-20,1e20,name='hd'),
#                 #Real(0.1,1,name='cfactor'),
#                 #Real(1e-6,1e-2,name='Csed'),
#                 Real(0.1,0.3,name = 'l1')]

# Define the objective function to be minimized
# @use_named_args(params)

params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'nP': 16645161.290322721,
 'q': 1.2874561841905887,
 'PSED': 1,
 'CburialP': 0.2,
 'CPsedEpi': 0.00010563582513371828,
 'CPsedHyp': 0.0035651695362787727,
 'l1': 0.3}
def PLGM(params):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        l1 = params0['l1'] 
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp*Tv - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp*Tv - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp*Tv - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   

    Psed[0] = (SedimentArea * SedimentPConc)*PSED*PSEDv
    
    for i in Ndays[1:]:
        # LoadTP = DF['PLoad[g]']*Lv
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1]*EntrPv # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1]*EntrPv # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(Oxv*DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.Kz[i-1]*kmzv#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]*Lv    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]*Lv        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth*Cburialv    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        DF0 = DF.copy()
        loss = own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
#         clear_output()
#         y_et = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
#         y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

#         y_et_pred = Pepi[:-2]
#         y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

#         y_ht = (DF['HypTP[g/m3]']*DF.HypV)[:-2]
#         y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

#         y_ht_pred = Phyp[:-2]
#         y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


#         r2_e    = r2_score(y_e,y_e_pred)
#         rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

#         r2_h    = r2_score(y_h,y_h_pred)
#         rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

#         loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)


        return(loss)
        #
    elif Output == 0 and not cond:
        loss = np.nan
        return(loss)
    else:
        if O =='df':
            DF0 = DF.copy()
            df = pd.DataFrame(data = {
                              'Time': Time[:13104],
                               'Season':  Season[:13104],
                              'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            if cond:
                DF0 = DF.copy()
                loss = own_metric(DF,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
            else:
                loss = np.nan
            return(df,loss)
        else:
            return(Pepi[:-2], Phyp[:-2])
        

    #return(df)

In [12]:
sen_n = ['kmzvi','PSEDvi','Cburialvi','EntrPvi','Lvi','Oxvi','Sedvi','Tvi','Evi']
sen_v = [[0,1.5,2,5,10,80,160,500,1000],   # Kmz
         [0,1.5,2,5,10],   # Psed
         [0,1.5,2,5,10],   # Cburialv
         [0,1.5,2,5,10],   # EntrPv
         [0,1.5,2,5,10],   # Pload
         [0,1.5,2,5,10],   # Oxv
         [0,1.5,2,5,10],   # Sedv
         [0,1.5,2,5,10],   # Tv
         [0,1.5,2,5,10]]   # Ev
for ix, i in enumerate(sen_n):
    exec(f'{i} = {sen_v[ix]}')


In [13]:
DF0 = DF.copy()
Output = 0; Data = 1
baseline_score = 0

Output = 0; Data = 1
for i in sen_n:
    exec(f'{i[:-1]} = {1}')
PLGM(params0)

NameError: name 'Psed' is not defined

In [71]:
#                                     ''' Mechanistic Model for the Mass Balance '''
# # To monitorize
# Ndays = range(0,len(DF.index),1)
# V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

# for v in V:
#     exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

# #
# '''Parameters Initialization'''

# # Calculate epi and hypo volume vectors, which should always add up to lake volume
# HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
# EpiV = np.asarray(DF.EpiV)

# # Change in epilimnetic volume (m3), used for entrainment
# # where a +change means transfer of volume from hypo to epi
# dEpiV = np.insert(np.diff(EpiV),0,0)

# Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
# Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 500,
 'CburialP': 0.01,
 'l1': 0.3
}
DF0 = DF.copy()
Output = 0; Data = 1
baseline_score = 0

Output = 0; Data = 1
for i in sen_n:
    exec(f'{i[:-1]} = {1}')

score = PLGM(params0)
baseline_score = score 

feature_importances = {}

overall_importances = {}
Seasonal_importances = {}

Output = 0; Data = 1
f = lambda l,v: [x for x in l if x != v]

for ix,i in enumerate(sen_n):
    for vx,v in enumerate(sen_v[ix]): 
        clear_output(wait = True)
        exec(f'{i[:-1]} = {v}')
        score = 0
        for rx,r in enumerate(f(sen_n,i)):
            exec(f'{r[:-1]} = {1}')           
        DF0 = DF.copy()
        Output = 0; Data = 1
        score +=  PLGM(params0)     
        print(score)
        if i not in feature_importances: 
            feature_importances[i] = {vx:np.abs(baseline_score - score)}
        else:
            feature_importances[i][vx] = np.abs(baseline_score - score)
        print(score)

KeyboardInterrupt: 

In [86]:
params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 500,
 'CburialP': 0.01,
 'l1': 0.3
}
Time = DF.Time
Season = DF.Season
Output = 1; Data = 1; O = 'df'
for i in sen_n:
    exec(f'{i[:-1]} = {1}')
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

In [84]:
params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 500,
 'CburialP': 0.01,
 'l1': 0.3
}
Time = DF.Time
Season = DF.Season
Output = 1; Data = 1; O = 'df'
for i in sen_n:
    exec(f'{i[:-1]} = {1}')
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

overall_baseline = score
for vx,v in enumerate(sen_v[ix]): 
    clear_output(wait = True)
    exec(f'{i[:-1]} = {v}')
    score = 0
    for rx,r in enumerate(f(sen_n,i)):
        exec(f'{r[:-1]} = {1}')            
    DF0 = DF.copy()
    Output = 0; Data = 1
    score +=  PLGM(params0)
    print(i , score, overall_baseline)

    if i not in overall_importances:             
        overall_importances[i] = {vx:np.abs(overall_baseline - score)}
    else:
        overall_importances[i][vx] = np.abs(overall_baseline - score)

    Output = 1; Data = 1; O = 'df'           
    pred = PLGM(params0)[0].set_index('Time')

KeyboardInterrupt: 

In [ ]:
                    
''' Mechanistic Model for the Mass Balance '''
# To monitorize
Ndays = range(0,len(DF0.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF0.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF0.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF0['EpiTP[g/m3]'][0]*DF0['EpiV'][0] 
Phyp[0] = DF0['HypTP[g/m3]'][0]*DF0['HypV'][0]

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 800,
 'CburialP': 0.01,
 'l1': 0.3
}

Output = 1; Data = 1; O = 'df'
for i in sen_n:
    exec(f'{i[:-1]} = 1')
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

overall_baseline = score 
seasonal_baseline = []

overall_importances = {}
seasonal_importances = {}

for ix, i in enumerate(pred.Season.unique()):
    
    pe = pred[pred.Season == i]\
                .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['EpiTP[g]']]\
                .agg(np.median).values.T[0]

    ph = pred[pred.Season == i]\
                .groupby([pred[pred.Season == i].index.year,pred[pred.Season == i].index.month])[['HypTP[g]']]\
                .agg(np.median).values.T[0]
    DF0 = DF.copy()
    seasonal_baseline.append(own_metric(DF0,Pepi = pe,Phyp = ph,l1= 0.3, t = 0,s= i))
    
f = lambda l,v: [x for x in l if x != v]

for ix,i in enumerate(sen_n):
    for vx,v in enumerate(sen_v[ix]): 
        clear_output(wait = True)
        exec(f'{i[:-1]} = {v}')
        score = 0
        for rx,r in enumerate(f(sen_n,i)):
            exec(f'{r[:-1]} = {1}')            
        DF0 = DF.copy()
        Output = 0; Data = 1
        score +=  PLGM(params0)
        print(i , score, overall_baseline)
        
        if i not in overall_importances:             
            overall_importances[i] = {vx:np.abs(overall_baseline - score)}
        else:
            overall_importances[i][vx] = np.abs(overall_baseline - score)
        
        Output = 1; Data = 1; O = 'df'           
        pred = PLGM(params0)[0].set_index('Time')
        for sx, s in enumerate(pred.Season.unique()):
            print('Seaon sensitive')
            score = 0
            pe = pred[pred.Season == s]\
                        .groupby([pred[pred.Season == s].index.year,pred[pred.Season == s].index.month])[['EpiTP[g]']]\
                        .agg(np.median).values.T[0]

            ph = pred[pred.Season == s]\
                        .groupby([pred[pred.Season == s].index.year,pred[pred.Season == s].index.month])[['HypTP[g]']]\
                        .agg(np.median).values.T[0]
            DF0 = DF.copy()
            if np.isnan(np.asarray(pe[:-2])).any():
                score = np.nan
            else:
                score = own_metric(DF0,Pepi = pe,Phyp = ph,l1= 0.3, t = 0,s= s)
             
            print(i, s, seasonal_baseline[sx],score)
            if s not in seasonal_importances:
                seasonal_importances[s]={i:{vx:np.abs(seasonal_baseline[sx] - score)}}
                
            else:
                if i not in seasonal_importances[s]:
                    seasonal_importances[s][i]= {vx:np.abs(seasonal_baseline[sx] - score)}                
                        
                else: 
                    seasonal_importances[s][i][vx]= np.abs(seasonal_baseline[sx] - score)
    

           

In [ ]:
pd.DataFrame(overall_importances)

In [ ]:
sa_s = pd.DataFrame(seasonal_importances)
for s in sa_s:
    eval
for c in sa_s.columns:
    exec(f'{c} = []')
    for ix,i in enumerate(sa_s.index):
        sa_s.loc[i,c]= np.nanmax(list(sa_s.loc[i,c].values()))
sa_s

In [ ]:
oa_s = pd.DataFrame(overall_importances)

oa_s.to_csv('PLGM_OSA_2.csv',encoding='utf-8')
sa_s.to_csv('PLGM_GSA_2.csv',encoding='utf-8')

In [ ]:
#oa_s = pd.DataFrame(overall_importances)

oa_s.to_csv('PLGM_OSA.csv',encoding='utf-8')
sa_s.to_csv('PLGM_GSA.csv',encoding='utf-8')

In [ ]:
pd.DataFrame(seasonal_importances)

In [ ]:
sa_s = pd.DataFrame(seasonal_importances)
for s in sa_s:
    eval
for c in sa_s.columns:
    exec(f'{c} = []')
    for ix,i in enumerate(sa_s.index):
        if i != 'kmzvi':
            sa_s.loc[i,c]= list(sa_s.loc[i,c].values())[0]
        else:
            sa_s.loc[i,c]= np.nanmax(list(sa_s.loc[i,c].values()))
sa_s.sort_values('autumn', ascending=False)

In [ ]:
most_important_variable = max(variable_scores, key=variable_scores.get)

In [ ]:
Output = 0; Data = 1
for i in sen_n:
    exec(f'{i[:-1]} = 1')

feature_importances = {}

overall_importances = {}
Seasonal_importances = {}
baseline_score = PLGM(params0)
print(baseline_score)
Output = 0; Data = 1
f = lambda l,v: [x for x in l if x != v]

for ix,i in enumerate(sen_n):
    for vx,v in enumerate(sen_v[ix]): 
        clear_output(wait = True)
        exec(f'{i[:-1]} = {v}')
        score = 0
        for rx,r in enumerate(f(sen_n,i)):
            exec(f'{r[:-1]} = {1}')          
        
        score +=  PLGM(params0)
        print(i , score, baseline_score)
        
        
        if i not in feature_importances: 
            feature_importances[i] = {vx:np.abs(baseline_score - score)}
        else:
            feature_importances[i][vx] = np.abs(baseline_score - score)

In [ ]:
 feature_importances

In [ ]:
feature_importances

In [ ]:
Output = 1; Data = 1
pred = PLGM(params0)[0] 
l = PLGM(params0)[1] 

In [ ]:
Output = 1; Data = 1
pred = PLGM(params0)[0] 
l = PLGM(params0)[1] 

t = y_true.groupby([y_true.index.year,y_true.index.month]).agg(np.median).values#.T[0]
p = pred.groupby([pred.index.year,pred.index.month]).agg(np.median).values#.T[0]
mep = np.median(fe(own_norm(t,inf = True),own_norm(p,inf = True)))#np.nanmedian((df_p-df_t)/df_t)#fe(df_t,df_p)
#mep = np.nanmedian(np.asarray(abs((own_norm(p))-np.asarray(own_norm(t))))/abs(np.asarray(own_norm(t))))#fe(own_norm(df_t),own_norm(df_p))
#q=quartiles(dt[0].values,dt[1].values); IQRp= q[0];CVp = q[1];  IQRt = q[2];CVt = q[3]
q=quartiles(y_true.values,pred.values)
IQRp= q[0];IQRt = q[2];  CVp = q[1];CVt = q[3]
IQRe = (IQRp-IQRt)/IQRt ; CVe = (CVp-CVt)/CVt

rmse = mean_squared_error(own_norm(y_true),own_norm(pred))**(1/2)
mse = mean_squared_error(own_norm(y_true),own_norm(pred))
mae = mean_absolute_error(own_norm(y_true),own_norm(pred))
r2 = r2_score(own_norm(y_true),own_norm(pred))

In [ ]:
feature_importances

In [ ]:
most_important_variable = max(feature_importance, key=variable_scores.get)
print("Most important variable:", most_important_variable)

#### Validation

In [ ]:
def PLGM(params):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        l1 = params0['l1'] 
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF.SedTemp*Tv - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF.EpiTemp*Tv - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF.HypTemp*Tv - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   

    Psed[0] = (SedimentArea * SedimentPConc)*PSED*PSEDv
    
    for i in Ndays[1:]:
        # LoadTP = DF['PLoad[g]']*Lv
        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1]*EntrPv # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1]*EntrPv # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(Oxv*DF['SedOx[g/m3]'][i-1]*DF.SedV[i-1] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.Strat[i-1] == False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.Kz[i-1]*kmzv#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.HypZm[i-1]-DF.EpiZm[i-1]) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.EpiZm[i-1]*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.HypZm[i-1]*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]*Sedv#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]*Sedv#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]*Lv    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]*Lv        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth*Cburialv    # Sedimnets burial
        
        ''' Update '''
        if DF.Strat[i-1] == False and DF.Strat[i] == False: # Full mixed
            
            VpEpi      = DF.EpiV[i-1]/(DF.EpiV[i-1]+DF.HypV[i-1])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        DF0 = DF.copy()
        loss = own_metric(DF0,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
#         clear_output()
#         y_et = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
#         y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

#         y_et_pred = Pepi[:-2]
#         y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

#         y_ht = (DF['HypTP[g/m3]']*DF.HypV)[:-2]
#         y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

#         y_ht_pred = Phyp[:-2]
#         y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


#         r2_e    = r2_score(y_e,y_e_pred)
#         rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

#         r2_h    = r2_score(y_h,y_h_pred)
#         rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

#         loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)


        return(loss)
        #
    elif Output == 0 and not cond:
        loss = np.nan
        return(loss)
    else:
        if O =='df':
            DF0 = DF.copy()
            df = pd.DataFrame(data = {
                              'Time': Time[:13104],
                               'Season':  Season[:13104],
                              'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            if cond:
                DF0 = DF.copy()
                loss = own_metric(DF,Pepi = Pepi,Phyp = Phyp,l1 = 0.3)
            else:
                loss = np.nan
            return(df,loss)
        else:
            return(Pepi[:-2], Phyp[:-2])
        

In [24]:

#                                     ''' Mechanistic Model for the Mass Balance '''
# #r2_e,r2_h,rmse_e,rmse_h,loss = [None]*5
# # ODE
# # To monitorize

DFF = pd.read_csv('DF.csv')
DF = DF.assign(SedTemp = DFF.SedTemp, 
               EpiTemp =DFF.EpiTemp, 
               HypTemp = DFF.HypTemp,
               Strat = DFF.Strat,
               EpiV  = DF['EpiV[L]'],
               HypV = DF['HypV[L]'],
               Kz   = DFF.Kz,
               HypZm = DFF.HypZm, 
               EpiZm = DFF.EpiZm,
               Time  = DFF.Time) 
DF['SedOx[g/m3]'] = DFF['SedOx[g/m3]']*1000
DF['SedV'] = DFF['SedV']

params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'nP': 16645161.290322721,
 'q': 1.2874561841905887,
 'PSED': 1,
 'CburialP': 0.2,
 'CPsedEpi': 0.00010563582513371828,
 'CPsedHyp': 0.0035651695362787727,
 'l1': 0.3}
params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 500,
 'CburialP': 0.01,
 'l1': 0.3
}
Time = DF.Time
Season = DF.Season
Output = 1; Data = 1; O = 'df'
pred = PLGM(params0)[0].set_index('Time')
score = PLGM(params0)[1]

C:\Users\gabri\AppData\Local\Temp\ipykernel_920\1408497805.py:118: RuntimeWarning: overflow encountered in scalar multiply
  Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])*Ev     # River outflow
C:\Users\gabri\AppData\Local\Temp\ipykernel_920\1408497805.py:140: RuntimeWarning: invalid value encountered in scalar add
  Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]


In [ ]:
overall_baseline = score
for vx,v in enumerate(sen_v[ix]): 
    clear_output(wait = True)
    exec(f'{i[:-1]} = {v}')
    score = 0
    for rx,r in enumerate(f(sen_n,i)):
        exec(f'{r[:-1]} = {1}')            
    DF0 = DF.copy()
    Output = 0; Data = 1
    score +=  PLGM(params0)
    print(i , score, overall_baseline)

    if i not in overall_importances:             
        overall_importances[i] = {vx:np.abs(overall_baseline - score)}
    else:
        overall_importances[i][vx] = np.abs(overall_baseline - score)

    Output = 1; Data = 1; O = 'df'           
    pred = PLGM(params0)[0].set_index('Time')

In [18]:
def own_metric(y_val,Pepi):
            clear_output()
#             y_et = (DF['EpiTP[g/m3]']*DF.EpiV)[:-2]
#             y_e = [(x - np.nanmin(y_et)) / (np.nanmax(y_et) - np.nanmin(y_et)) for x in y_et]

#             y_et_pred = Pepi[:-2]
#             y_e_pred = [(x - np.nanmin(y_et_pred)) / (np.nanmax(y_et_pred) - np.nanmin(y_et_pred)) for x in y_et_pred]

#             y_ht = (DF['HypTP[g/m3]']*DF.HypV)[:-2]
#             y_h = [(x - np.nanmin(y_ht)) / (np.nanmax(y_ht) - np.nanmin(y_ht)) for x in y_ht]    

#             y_ht_pred = Phyp[:-2]
#             y_h_pred = [(x - np.nanmin(y_ht_pred)) / (np.nanmax(y_ht_pred) - np.nanmin(y_ht_pred)) for x in y_ht_pred]


#           r2_e    = r2_score(y_e,y_e_pred)
#           rmse_e  = np.sqrt(mean_squared_error(y_e,y_e_pred))

            # r2_h    = r2_score(y_h,y_h_pred)
            # rmse_h  = np.sqrt(mean_squared_error(y_h,y_h_pred))

            # 
            # l2 = 1 - l1
            # l1e = l2*0.3 
            # l2h = l2-l1e
            r2 = r2_score(y_val,Pepi)
            rmse = np.sqrt(mean_squared_error(y_val,Pepi))
            loss = r2*0.5 + rmse*0.5
            # loss = rmse_e*l1e + rmse_h*l2h + abs(1-r2_e)*(l1e) + abs(1-r2_h)*(l2h)
            #loss = rmse_e + rmse_h + abs(1-r2_e) + abs(1-r2_h)

            # print('-----Results-----')
            # print(f'R2=>  Epi: {r2_e:.2%}  Hyp: {r2_h:.2%}')
            # print(f'rmse=>  Epi: {rmse_e:.4}  Hyp: {rmse_h:.4}')
            # print(f'loss: {loss:.4}\n')
            return(loss)

In [23]:
                                    ''' Mechanistic Model for the Mass Balance '''
# To monitorize
# DF = pd.read_csv('plgm.csv')
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF['HypV[L]'])# hypolimnetic volume time series
EpiV = np.asarray(DF['EpiV[L]'])

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF['EpiP[ug]'][0]*DF['HypV[L]'][0] 
Phyp[0] = DF['HypP[ug]'][0]*DF['EpiV[L]'][0]

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 1000,
 'CburialP': 0.01,
 'l1': 0.3
}
Output = 1; Data = 1; O = 'df'
plgm = PLGM(params0)
num = plgm.shape[0]

plgm = plgm.assign(Time = DF.Time[:num]
                   ,Season =  DF.Season[:num]
                   ,PEPItrue = DF['EpiP[ug]'][:num].values*DF['EpiV[L]'][:num].values
                   ,HYPtrue  =DF['HypP[g/m3]'][:num].values*DF['HypV[L]'][:num].values )\
            .rename(columns = {'EpiP[g]':'MMpred'})
#DF.loc[:num-1,'EpiTP[g/m3]'].values*DF.loc[:num-1,'EpiV'].values)
#plgm.to_csv('plgm.csv')

AttributeError: 'tuple' object has no attribute 'assign'

####

In [ ]:
@use_named_args(params)
def PLGM(**params):
    
    '''
    Output  : [0] Array [ELSE] DataFrame
    Data    : [0] Optimization [ELSE] fix values
    
    '''
    #DF = DF.copy()
    if Data == 0:
        Fp  = params['Fp'];ThetaRp = params['ThetaRp'];ThetaSPhyp = params['ThetaSPhyp'];ThetaSPepi  = params['ThetaSPepi']
        KsedP  = params['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params['TbaseSPepi']; TbaseSPhyp  = params['TbaseSPhyp']; TbaseRp = params['TbaseRp']
        RrecP = params['RrecP'];  PSED = params['PSED'];
        #nP = params['nP'];q = params['q']
        CburialP =params['CburialP']#;CPsedHyp = params['CPsedHyp'];CPsedEpi = params['CPsedEpi']
        #ed =params['ed'];hd=params['hd'];cfactor=params['cfactor']
        #;Csed = params['Csed']
        l1 = params['l1'] 
        l2 = 1 - l1
        l1e = l2*0.3 
        l2h = l2-l1e
        loss_best = 1000000
    else:        
        Fp  = params0['Fp'];ThetaRp = params0['ThetaRp'];ThetaSPhyp = params0['ThetaSPhyp'];ThetaSPepi  = params0['ThetaSPepi']
        KsedP  = params0['KsedP']; SedDepth = 0.1
        TbaseSPepi  = params0['TbaseSPepi']; TbaseSPhyp  = params0['TbaseSPhyp']; TbaseRp = params0['TbaseRp']
        RrecP = params0['RrecP'];  PSED = params0['PSED'];
        #nP = params0['nP'];q = params0['q']
        CburialP =params0['CburialP']
        ##CPsedHyp = params0['CPsedHyp'];CPsedEpi = params0['CPsedEpi']
    

        
    CoefRp       = np.asarray(ThetaRp**(DF0.SedTemp - TbaseRp)) # Arrhenius coefficient for recycling
    CoefSPepi    = np.asarray(ThetaSPepi**(DF0.EpiTemp - TbaseSPepi)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
    CoefSPhyp    = np.asarray(ThetaSPhyp**(DF0.HypTemp - TbaseSPhyp)) # Arrhenius coefficient for sedimentation (assuming phytos are sinking)
   

    Psed[Ndays[0]] = (SedimentArea * SedimentPConc)*PSED
    
    for i in Ndays[1:]:

        '''Entrainment'''

        if dEpiV[i] == 0:        
            EntrP[i] = 0 # Phosphorus entrainment
            #EntrN[i] = 0 # Nitrogen entrainmnet 

        elif dEpiV[i] >0:        
            EntrP[i] = (dEpiV[i]/HypV[i-1])*Phyp[i-1] # gP from Hyp
            #EntrN[i] = (dEpiV[i]/HypV[i-1])*Nhyp[i-1] # gN from Hyp
        
        else:
            EntrP[i] = (dEpiV[i]/EpiV[i-1])*Pepi[i-1] # gP from Pepi
            #EntrN[i] = (dEpiV[i]/EpiV[i-1])*Nepi[i-1] # gN from Pepi

        '''Recycling''' 
        Rp[i] = Psed[i-1]*(RrecP)*(KsedP/(DF.loc[i-1,'SedOx[g/m3]']*DF.loc[i-1,'SedV'] + KsedP))*CoefRp[i-1] #*((Psed[i-1]**q)/((Psed[i-1]**q)+(nP**q)))
        Hvd[i] = 0
        '''Vertical Diffusion (Hvd)'''
        if DF.loc[i-1,'Strat']== False:
            Csed = 14.5/(86000000000*155*365)
            CPsedEpi = Csed/2
            CPsedHyp = Csed/2
        else:
            kmz = DF.loc[i-1,'Kz']*0#0.005*1.6*(151**0.568) # Diffusion coeficient
            #Hvd[i] = DF.Am[i-1]*kmz*(Phyp[i-1]/HypV[i-1]-Pepi[i-1]/EpiV[i-1])/(HypZm[i-1]-EpiZm[i-1])# Diffusion
            Hvd[i] = kmz*(Phyp[i-1]-Pepi[i-1])/(DF.loc[i-1,'HypZm']-DF.loc[i-1,'EpiZm']) # Diffusion
            CPsedEpi = (14.5/(EpiV[i-1]*DF.loc[i-1,'EpiZm']*365))
            CPsedHyp = (14.5/(HypV[i-1]*DF.loc[i-1,'HypZm']*365))            

       
        '''Sedimentation'''

        SedPepi[i] = CPsedEpi*Pepi[i-1]*CoefSPepi[i-1]#*((Pepi[i-1]**qE)/((Pepi[i-1]**qE)+(nPE**qE)))   # Sed's phosphorus,
        SedPhyp[i] = CPsedHyp*Phyp[i-1]*CoefSPhyp[i-1]#*((Phyp[i-1]**qH)/((Phyp[i-1]**qH)+(nPE**qH)))   # Sed's phosphorus,CPsedHyp

        '''Loss'''
        # Nutrients
        Ep[i]    = Qout[i-1]*(Pepi[i-1]/EpiV[i-1])     # River outflow
        
        '''Input'''
        #Phosphorus
        IepiP[i] = (1-Fp)*LoadTP[i-1]    # To epiliminion
        IsedP[i] = Fp*LoadTP[i-1]        # To sediment
        '''Burial'''
        Bp[i] = CburialP*(1/365)*(1/1000)*Psed[i-1]/SedDepth    # Sedimnets burial
        
        ''' Update '''
        if DF.loc[i-1,'Strat'] == False and DF.loc[i,'Strat'] == False: # Full mixed
            
            VpEpi      =DF.loc[i-1,'EpiV']/(DF.loc[i-1,'EpiV']+DF.loc[i-1,'HypV'])
            
            EntrP[i]   = 0
            Hvd[i]     = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi       +  Rp[i]*VpEpi      + IepiP[i]    - SedPepi[i]  - Ep[i]     
            Phyp[i] = (Pepi[i-1]+Phyp[i-1])*(1-VpEpi)   +  Rp[i]*(1-VpEpi)  + IsedP[i]    -SedPhyp[i] 
            Psed[i] =  Psed[i-1] + SedPhyp[i]  + SedPepi[i]  -  Rp[i]  - Bp[i]

        else:
            Pepi[i] = Pepi[i-1] + IepiP[i]    + EntrP[i]  + Hvd[i] - SedPepi[i]  - Ep[i]  
            Phyp[i] = Phyp[i-1] + Rp[i]       + IsedP[i]  - Hvd[i] + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i]     - Bp[i]
    
    cond = not np.isnan(np.asarray(Pepi[:-2])).any()   
    #cond1 = not np.asarray([True if i is None else False for i in Pepi[:-2]]).any()
    time.sleep(1.5)
    clear_output(wait = True)
    if Output == 0 and cond:
        if train:
            o = own_metric(y_val[4:],Pepi[Ndays[1]:])
        else:
            o = own_metric(y_val[4:],Pepi[Ndays[1]:])
        return()
        #
    elif Output == 0 and not cond:
        loss = random.uniform(10, 15)
        return(loss)
    else:
        if O =='df':
            df = pd.DataFrame(data = {'EpiTP[g]':list(Pepi[:-2]),#/np.asarray(DF.EpiV[:13106])
                              'EntrTP[g]':list(EntrP[:-2]),
                              'IepiP': list(IepiP[:-2]),
                              'SedPepi[g]':list(SedPepi[:-2]),
                              'Ep[g]':list(Ep[:-2]),
                              'Hvd':list(Hvd[:-2]), 
                              'HypTP[g]':list(Phyp[:-2]),#/np.asarray(DF.HypV[:13106])
                              'SedPhyp[g]':list(SedPhyp[:-2]),
                              'Rp[g]':list(Rp[:-2]),
                              'IsedP[g]':list(IsedP[:-2]),
                              'Psed[g]':list(Psed[:-2]),
                              'Bp[g]':list(Bp[:-2])

                             })
            return(df)
        else:
            return Pepi[Ndays[0]:]#(Pepi[:-2], Phyp[:-2])
        

    #return(df)

In [ ]:
DF0 = pd.read_csv('DF.csv', sep = ',')
DF0['Time'] = pd.to_datetime(DF0.Time)
DF0.columns
DF0.replace(np.nan, 0, inplace = True)

In [ ]:
# Step 1: Split the dataset
DF_train, DF, y_train, y_val = train_test_split(DF0.iloc[:,1:], DF0['EpiTP[g/m3]']*DF0['EpiV'] , test_size=0.2, random_state=10,shuffle = False)

# Step 2: Fit the model

LoadTP = DF['PLoad[g]']
LoadTN = DF['NLoad[g]']
a_ratio  = 0.5   # Ratio C:Chla content' 
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

# Step 3: Predict on the validation set
''' Mechanistic Model for the Mass Balance '''
# To monitorize
Ndays = range(DF.loc[(DF.index[0]):,:].index[0]+1,DF.loc[(DF.index[0]):,:].index[-1],1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(DF0.index))') #len(Ndays)

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = DF['HypV']# hypolimnetic volume time series
EpiV = DF['EpiV']

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = pd.Series(data = np.insert(np.diff(EpiV),0,0),index = EpiV.index)

Pepi[Ndays[0]] = DF.loc[DF.index[0],'EpiTP[g/m3]']*DF.loc[DF.index[0],'EpiV'] 
Phyp[Ndays[0]] = DF.loc[DF.index[0],'HypTP[g/m3]']*DF.loc[DF.index[0],'HypV']

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 1000,
 'CburialP': 0.01,
 'l1': 0.3
}

Output = 1; Data = 1; O = None
y_pred = PLGM(params0)

# Step 4: Assess outlier performance

residuals = np.abs(own_norm(y_pred) - own_norm(y_val[1:]))  # Calculate the absolute residuals
threshold =np.abs(stats.zscore(residuals))
outliers = np.where(residuals > threshold)  # Define outliers based on a threshold

# Step 5: Evaluate overfitting


In [ ]:
# Step 1: Split the dataset
DF_train, DF, y_train, y_val = train_test_split(DF0.iloc[:,1:], DF0['EpiTP[g/m3]']*DF0['EpiV'] , test_size=0.2, random_state=10,shuffle = False)
# Step 3: Predict on the validation set
DF = DF_train

LoadTP = DF['PLoad[g]']
LoadTN = DF['NLoad[g]']
a_ratio  = 0.5   # Ratio C:Chla content' 
## Lake 
#t = 11.5 # Residencial time (theretical or constant?) (years)
TOC = 1.1 # %
#Zmean = 151.7 # m
LakeArea = 582210000 # m2
#V = 86000000000 # m3
Qout = DF['Qout[m3/d]']
#sc_strat = 0.5


# Sediment 
AvailableP = 0.65 # mg of P/g od fry weight
AvailableN = 200  # mg of N/g od fry weight
density = 1330000.0 #g/m^3 googled standard value for silt/clay
SedDepth = 0.1
SedimentArea  = LakeArea # [m2]
SedimentVolume = SedimentArea * SedDepth #m3 # 41.90e7
SedimentPConc = AvailableP * density * (1/1000) *SedDepth # g/m2 of sedi
SedimentNConc = AvailableN * density * (1/1000) *SedDepth # g/m2 of sedi

''' Mechanistic Model for the Mass Balance '''
# To monitorize
Ndays = range(DF.loc[(DF.index[0]):,:].index[0]+1,DF.loc[(DF.index[0]):,:].index[-1],1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(DF0.index))') #len(Ndays)

#
'''Parameters Initialization'''


# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = DF['HypV']# hypolimnetic volume time series
EpiV = DF['EpiV']

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = pd.Series(data = np.insert(np.diff(EpiV),0,0),index = EpiV.index)

Pepi[Ndays[0]] = DF.loc[DF.index[0],'EpiTP[g/m3]']*DF.loc[DF.index[0],'EpiV'] 
Phyp[Ndays[0]] = DF.loc[DF.index[0],'HypTP[g/m3]']*DF.loc[DF.index[0],'HypV']

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 1000,
 'CburialP': 0.01,
 'l1': 0.3
}
y_pred_train = PLGM(params0)[:10484]
v1 = list(own_norm(y_train))
v2 = list(own_norm(y_pred_train))
residuals_train = [abs(a - b) for a, b in zip(v1, v2)]
threshold =np.abs(stats.zscore(residuals_train ))
outlier_ratio_train = len(np.where(residuals_train > threshold)[0]) / len(y_train)
outlier_ratio_val = len(outliers[0]) / len(y_val)

# Print the outlier ratios for comparison
print("Outlier ratio (training set):", outlier_ratio_train)
print("Outlier ratio (validation set):", outlier_ratio_val)

#### PLotting

In [ ]:
                                    ''' Mechanistic Model for the Mass Balance '''
# To monitorize
Ndays = range(0,len(DF.index),1)
V = ('EntrP,Rp,SedPepi,SedPh,Ep,IepiP,IsedP,Bp,SedPhyp,Hvd,Pepi,Phyp,Psed').split(',')

for v in V:
    exec(f'{v} = np.asarray([np.nan]*len(Ndays))') 

#
'''Parameters Initialization'''

# Calculate epi and hypo volume vectors, which should always add up to lake volume
HypV = np.asarray(DF.HypV)# hypolimnetic volume time series
EpiV = np.asarray(DF.EpiV)

# Change in epilimnetic volume (m3), used for entrainment
# where a +change means transfer of volume from hypo to epi
dEpiV = np.insert(np.diff(EpiV),0,0)

Pepi[0] = DF['EpiTP[g/m3]'][0]*DF['EpiV'][0] 
Phyp[0] = DF['HypTP[g/m3]'][0]*DF['HypV'][0]

params0 = {
 'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'PSED': 800,
 'CburialP': 0.01,
 'l1': 0.3
}
Output = 1; Data = 1; O = 'df'
plgm = PLGM(params0)
num = plgm.shape[0]
plgm = plgm.assign(Time = DF.Time[:num]
                   ,Season =  DF.Season[:num]
                   ,PEPItrue = DF['EpiTP[g/m3]'][:num].values*DF['EpiV'][:num].values
                   ,HYPtrue  =DF['HypTP[g/m3]'][:num].values*DF['HypV'][:num].values )\
            .rename(columns = {'EpiTP[g]':'MMpred'})
#DF.loc[:num-1,'EpiTP[g/m3]'].values*DF.loc[:num-1,'EpiV'].values)
plgm.to_csv('plgm.csv')

In [ ]:
p = 'MMpred'
pt = 'PEPItrue'
#s = 'summer'
maskp= plgm.Season.values == 'summer'
maskt= plgm.Time.isin(plgm[maskp].Time.values)

vp = plgm[maskp][p].values
vt = plgm[maskp][pt].values#pl[maskt][pt].values*DF[maskt]['EpiV'].values
pred = [(x - np.nanmin(vp)) / (np.nanmax(vp) - np.nanmin(vp)) for x in vp]

y_true = [(x - np.nanmin(vt)) / (np.nanmax(vt) - np.nanmin(vt)) for x in (vt)]

print(f'\nr2: {r2_score(y_true,pred):.2%}\
       \nrmse:{np.sqrt(mean_squared_error(y_true,pred)):.4}\
       \nmse:{mean_squared_error(y_true,pred):.4},\
       \nmae: {mean_absolute_error(y_true,pred):.4}')

In [ ]:
p = 'MMpred'
pt = 'PEPItrue'
#s = 'summer'
maskp= plgm.Season.values == 'summer'
maskt= plgm.Time.isin(plgm[maskp].Time.values)

vp = plgm[maskp][p].values
vt = plgm[maskp][pt].values#pl[maskt][pt].values*DF[maskt]['EpiV'].values
pred = [(x - np.nanmin(vp)) / (np.nanmax(vp) - np.nanmin(vp)) for x in vp]

y_true = [(x - np.nanmin(vt)) / (np.nanmax(vt) - np.nanmin(vt)) for x in (vt)]

print(f'\nr2: {r2_score(y_true,pred):.2%}\
       \nrmse:{np.sqrt(mean_squared_error(y_true,pred)):.4}\
       \nmse:{mean_squared_error(y_true,pred):.4},\
       \nmae: {mean_absolute_error(y_true,pred):.4}')

In [ ]:
k = 20
#k +=1
#params0 = {param.name: value for param, value in zip(params, result.x)}
n = 365*k

m = n+365
fig, axs = plt.subplots(1, 3,figsize=(30,8))
axs = axs.ravel()

# Epilimnion
axs[0].plot(DF.Time[n:m],(DF['EpiTP[g/m3]']*DF.EpiV)[n:m], label = 'y_true')
axs[0].plot(DF.Time[n:m],plgm['MMpred'][n:m], label = 'y_pred')
r2=r2_score((DF['EpiTP[g/m3]']*DF.EpiV)[n:m],plgm['MMpred'][n:m])
axs[0].text(DF.Time[n:m][int((n+m)/2)],1e11,f" R2: {r2:.2%}", ha="center", va="center", size=20)
axs[0].set_title('Epilimnion')

# Hypolimnion
axs[1].plot(DF.Time[n:m],(DF['HypTP[g/m3]']*DF.HypV)[n:m],label ='y_true')
axs[1].plot(DF.Time[n:m],plgm['HypTP[g]'][n:m], label = 'y_pred')
axs[1].set_title('Hypolimnion')
r2=r2_score((DF['HypTP[g/m3]']*DF.HypV)[n:m],plgm['HypTP[g]'][n:m])
axs[1].text(DF.Time[n:m][int((n+m)/2)],plgm['HypTP[g]'][n+30],f" R2: {r2:.2%}", ha="center", va="center", size=20)

# Sediment
axs[2].plot(DF.Time[n:m],plgm['Psed[g]'][n:m],label ='y_predicted')
axs[2].set_title('Sediment')


axs[0].set_ylabel('Total Phosphorus [g]')

axs[0].legend()
axs[1].legend()
axs[2].legend()

In [ ]:
k = 20
#k =20
#params0 = {param.name: value for param, value in zip(params, result.x)}
n = 365*k

m = n+365
fig, axs = plt.subplots(1, 3,figsize=(30,8))
axs = axs.ravel()

# Epilimnion
axs[0].plot(DF.Time[n:m],(DF['EpiTP[g/m3]']*DF.EpiV)[n:m], label = 'y_true')
axs[0].plot(DF.Time[n:m],plgm['EpiTP[g]'][n:m], label = 'y_pred')
r2=r2_score((DF['EpiTP[g/m3]']*DF.EpiV)[n:m],plgm['EpiTP[g]'][n:m])
axs[0].text(DF.Time[n:m][int((n+m)/2)],1e11,f" R2: {r2:.2%}", ha="center", va="center", size=20)
axs[0].set_title('Epilimnion')

# Hypolimnion
axs[1].plot(DF.Time[n:m],(DF['HypTP[g/m3]']*DF.HypV)[n:m],label ='y_true')
axs[1].plot(DF.Time[n:m],plgm['HypTP[g]'][n:m], label = 'y_pred')
axs[1].set_title('Hypolimnion')
r2=r2_score((DF['HypTP[g/m3]']*DF.HypV)[n:m],plgm['HypTP[g]'][n:m])
axs[1].text(DF.Time[n:m][int((n+m)/2)],plgm['HypTP[g]'][n+30],f" R2: {r2:.2%}", ha="center", va="center", size=20)

# Sediment
axs[2].plot(DF.Time[n:m],plgm['Psed[g]'][n:m],label ='y_predicted')
axs[2].set_title('Sediment')


axs[0].set_ylabel('Total Phosphorus [g]')

axs[0].legend()
axs[1].legend()
axs[2].legend()

In [ ]:
params0 = {'ThetaSPepi': 1.3123834064195374,
 'ThetaSPhyp': 1.1241227447142852,
 'TbaseSPepi': 4.113509201569038,
 'TbaseSPhyp': 14.179827943922088,
 'Fp': 0.24687844315278318,
 'TbaseRp': 7.902335061473191,
 'ThetaRp': 1.1264026422029476,
 'KsedP': 1613746294.5171556,
 'RrecP': 1e-05,
 'nP': 16645161.290322721,
 'q': 1.2874561841905887,
 'PSED': 1,
 'CburialP': 0.2,
 'CPsedEpi': 0.00010563582513371828,
 'CPsedHyp': 0.0035651695362787727,
 'l1': 0.3}

#### Sediment

In [ ]:
plt.plot(plgm['Psed[g]'])

In [ ]:
params0['nP'],params0['q']

In [ ]:
max(Psed)

In [ ]:
Psed = plgm['Psed[g]']
k = -np.sort(-Psed)#DF['EpiTP[g/m3]']*DF.EpiV)
nP= 0.07#params0['nP']#434009137.99901134 * (np.nanmax(Psed) - np.nanmin(Psed)) + np.nanmin(Psed)
q= 2#params0['q']
#q =1

s = 1
o = []
for n in k:
    o.append(s*( n**q )/(n**q + nP**q))
plt.plot(k,o)

In [ ]:
k = -np.sort(-DF['EpiTP[g/m3]']*DF.EpiV)#-np.sort(-PLGM(params,DF).iloc[:,-2])
#
mask_min = np.where(DF['EpiTP[g/m3]']==np.min(DF['EpiTP[g/m3]']))[0][0]
mask_max = np.where(DF['EpiTP[g/m3]']==np.max(DF['EpiTP[g/m3]']))[0][0]
np_min = (DF['EpiTP[g/m3]'][mask_min]*DF.EpiV[mask_min])*2
np_max = (DF['EpiTP[g/m3]'][mask_max]*DF.EpiV[mask_max])*10
np_mean = (np_min+np_max)/2
#
m = np_min#np.max(PLGM(params,DF).iloc[:,-2])/2
o = []
q =2
for n in k:
    o.append(( n**q )/(n**q + m**q))
plt.plot(k,o)

In [ ]:
print(f'{n:.2e}')

#### Hypolimnion

In [ ]:
a = PLGM(params,DF)
a['Time'] = DF.Time[:13106]
b = a[['Time','HypP[g]']]
c = pd.DataFrame(data ={'HypP[g]':DF['HypTP[g/m3]'][:13106]*DF.EpiV[:13106], 'HypP[g]':DF['HypTP[g/m3]'][:13106]*DF.HypV[:13106],'Time':a.Time})


In [ ]:
plt.figure(figsize=(20,10))
plt.plot(b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')].Time,b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')]['HypP[g]'], label = 'y_pred'),
plt.plot(b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')].Time,c[(c.Time >= f'{y}-01-01') & (c.Time <= f'{y1}-12-31')]['HypP[g]'], label = 'y_true')
plt.xlabel('Year')
plt.ylabel('Total Phosphorus')
plt.title('Hypolimnion')
plt.legend();

In [ ]:
b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')].Time,b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')]['EpiP[g]'], label = 'Y_pred'
plt.plot(

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(DF.Time[:13106],PLGM(params,DF).iloc[:,4], label = 'y_pred'),lt.plot(DF.Time[:13106],c['HypP[g]'], label = 'y_true')
plt.xlabel('Year')
plt.ylabel('Total Phosphorus')
plt.title('Hypolimnion')
plt.legend();

##### Epilimnion

In [ ]:
a = PLGM(params,DF)
a['Time'] = DF.Time[:13106]
b = a[['Time','EpiP[g]']]
c = pd.DataFrame(data ={'EpiP[g]':DF['EpiTP[g/m3]'][:13106]*DF.EpiV[:13106], 'HypP[g]':DF['HypTP[g/m3]'][:13106]*DF.HypV[:13106],'Time':a.Time})
c

In [ ]:
plt.figure(figsize=(20,10))
#c = pd.DataFrame(data ={'EpiP[g]':DF['EpiTP[g/m3]'][:13106]*DF.EpiV[:13106], 'HypP[g]':DF['HypTP[g/m3]'][:13106]*DF.HypV[:13106],'Time':a.Time})
plt.plot(DF.Time[0:50],PLGM(params,DF).iloc[:,0][0:50]),plt.plot(DF.Time[0:50],(DF['EpiP[g/m3]']*DF.EpiV)[0:50])#13106])

In [ ]:
plt.figure(figsize=(20,10)) # ,
y = 2009
y1 = 2010
plt.plot(b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')].Time,b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')]['EpiP[g]'], label = 'Y_pred'),plt.plot(b[(b.Time >= f'{y}-01-01') & (b.Time <= f'{y1}-12-31')].Time,c[(c.Time >= f'{y}-01-01') & (c.Time <= f'{y1}-12-31')]['EpiP[g]'],label = 'Y_true')
plt.legend();

In [ ]:
r2_score(PLGM(params,DF)[0],(DF['EpiTP[g/m3]'][:13106]*DF.EpiV[:13106]))

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(DF.Time,DF['EpiTP[g/m3]']*DF.EpiV)
plt.xlabel('Year')
plt.ylabel('Total Phosphorus [g]')
plt.title('Epilimnetic')

# Debbuging 

## .

In [ ]:
DO = DF['SedOx[g/m3]']#*DF.SedV
# Varibales
Phyp = DF['HypP[g/m3]']*DF['EpiV']
Nhyp = DF['HypN[g/m3]']*DF['HypV']
Pepi = DF['EpiP[g/m3]']*DF['EpiV']
Nepi = DF['EpiN[g/m3]']*DF['HypV']
phy = DF['Chla[g]']*0.5
#
Rp = np.asarray([np.nan]*len(Ndays))
Rn = np.asarray([np.nan]*len(Ndays))
cfsedN = np.asarray([np.nan]*len(Ndays))

Psed = DF['SedP[g/m3]']#np.asarray([SedimentArea * SedimentPConc]*len(Ndays))
Nsed = DF['SedN[g/m3]']#np.asarray([SedimentArea * SedimentNConc]*len(Ndays))

In [ ]:
#FsedN = np.asarray([np.nan]*len(Ndays))
#FsedP= np.asarray([np.nan]*len(Ndays))
#
Phy = np.asarray([np.nan]*len(Ndays))
Phy[0] = phy[8]
for i in Ndays[1:]:
    
    '''Pythoplankton'''
    Nlim[i] = Nepi[i-1]/(Nepi[i-1]+Khn[i]) 
    Plim[i] = Pepi[i-1]/(Pepi[i-1]+Khp[i]) 
    #Llim[i] = (Ipar/Iopt)*exp(1-(Ipar/Iopt))
    
    Gr[i] = Phy[i-1]*(umax*min(Nlim[i],Plim[i])*CoefPhy[i-1]) #,Llim
    
    PcPhy[i] = lamCP*Gr[i-1]                       # Phy uptake [i]
    NcPhy[i] = lamCN*Gr[i-1]
    
    #Phy Export, mortality, sink, and grazing
    Ephy[i] = Qout[i-1]*(Phy[i-1]/EpiV[i-1])    # Phy's river outflow
    MR[i]   = Phy[i-1]*Mmax*CoefPhy[i-1]        # Mortality rate
    Sr[i]   = Phy[i-1]*(u/Zmean)                # Sinking phy
    G[i]    = gp*(Phy[i-1]**2)                  # Grazing
    
    Phy[i]  = Gr[i]     - MR[i]    - Sr[i]     - G[i]    - Ephy[i]



In [ ]:
V = [Phy, Gr, MR, Sr, G, Ephy]

for v in V:
    plt.plot(Phy)

In [ ]:
Mmax,u/Zmean,umax

## Compartment

In [ ]:
i = 5

### Epiliminnion

In [ ]:
Pepi[i-1], IepiP[i], EntrP[i] , SedP[i], Ep[i], PcPhy[i]

### Hypolimnion

In [ ]:
Phyp[i-1], Rp[i]   , EntrP[i]

### Sediment

In [ ]:
Psed[i-1], SedP[i] , Rp[i]    , IsedP[i]  , Bp[i]

## Conditional statment

### Alternative:

In [ ]:
        Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]*SC  - SedP[i]  - Ep[i] #- PcPhy[i]
        Phyp[i] = Phyp[i-1] + Rp[i]    - EntrP[i]*SC  - SedP[i] + IsedP[i]
        Psed[i] = Psed[i-1] + SedP[i]  - Rp[i]        - Bp[i]    + SedP[i]

In [ ]:
        
        st = [1e-10 if DF['St'][i] <0 else DF['St'][i]][0]

        if (DF.Strat[i-1] == False and DF.Strat[i] == False):
            
            SC = [sc_winter if DF.Season[i] == 'winter' else sc_nonstrat][0]
        else:
            SC = (0.20*st/15) 

### Original

In [ ]:
# Ojo QUE ABAJAO dice (Phyp[i-1] Phyp[i-1])*(1-VpEpi) 

In [ ]:
        
        if DF['Strat'][i-1] == False and DF['Strat'][i] == False:
            
            VpEpi = DF.EpiV[i]/(DF.EpiV[i]+DF.HypV[i])
            EntrP[i] = 0
            #Rp[i] = 0
            
            Pepi[i] = (Pepi[i-1]+Phyp[i-1])*VpEpi +  Rp[i]*VpEpi + IepiP[i] + EntrP[i]  - SedPepi[i]  - Ep[i] #- PcPhy[i]
            Phyp[i] = (Phyp[i-1]+Phyp[i-1])*(1-VpEpi) + Rp[i]*(1-VpEpi)    + IsedP[i]     + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i] - Bp[i]
            
        elif DF['Strat'][i-1] == True and DF['Strat'][i] == True:
            Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]  - SedPepi[i]  - Ep[i] #- PcPhy[i]
            Phyp[i] = Phyp[i-1] + Rp[i]    + IsedP[i]     + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i] - Bp[i]
            
        else:
            #EntrP[i] =0
            Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]  - SedPepi[i]  - Ep[i] #- PcPhy[i]
            Phyp[i] = Phyp[i-1] + Rp[i]    + IsedP[i]     + SedPepi[i]  - EntrP[i]  - SedPhyp[i]
            Psed[i] = Psed[i-1] + SedPhyp[i]  - Rp[i] - Bp[i]   

In [ ]:
   '''Conditional statment'''
    
    if (Strat[i-1] == True and Strat[i] == False): # Fall turnover  
        EntrP[i] = 0
        EntrN[i] = 0
       
        
    elif (Strat[i-1] == False and Strat[i] == True): # Spring Stratification
        EntrP[i] = 0
        EntrN[i] = 0
          
        
    elif Strat[i-1] == False and Strat[i] == False: # Winter 
        # Entr != 0 because lost of stratification
        # 
        pass      
        
    elif (Strat[i-1] == True and Strat[i] == True): # Summer
        EntrP[i] = 0
        EntrN[i] = 0

        
        Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]  - SedP[i] - Ep[i] - PcPhy[i]
        Phyp[i] = Phyp[i-1] + Rp[i]    - EntrP[i]
        Psed[i] = Psed[i-1] + SedP[i]  - Rp[i]     + IsedP   - Bp[i]
        
        Nepi[i] = Nepi[i-1] + IepiN[i] + EntrN[i]  - SedN[i] - En[i] - NcPhy[i]
        Nhyp[i] = Nhyp[i-1] + Rn[i]    - EntrN[i]
        Nsed[i] = Nsed[i-1] + SedN[i]  - Rn[i]     + IsedN   - Bn[i] 
        
        Phy[i]  = Gr[i]     - Mr[i]    - Sr[i]     - G[i]    - Ephy[i]

In [ ]:
    if DF['Season'][i] == 'winter':
        if DF['St'][i] <0:
            st = 0            
        else:
            st = DF['St'][i]
        
        SC = 1-(0.2*st/15)
        perc.append(SC)
        Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]*SC  - SedP[i] - Ep[i] #- PcPhy[i]
        Phyp[i] = Phyp[i-1] + Rp[i]    - EntrP[i]*SC
        Psed[i] = Psed[i-1] + SedP[i]  - Rp[i]     + IsedP[i]   - Bp[i]

        Nepi[i] = Nepi[i-1] + IepiN[i] + EntrN[i]*SC  - SedN[i] - En[i]- DenitN[i]# - NcPhy[i] 
        Nhyp[i] = Nhyp[i-1] + Rn[i]    - EntrN[i]*SC
        Nsed[i] = Nsed[i-1] + SedN[i]  - Rn[i]     + IsedN[i]   - Bn[i] 
        #Phy[i]  = Gr[i]     - MR[i]    - Sr[i]     - G[i]    - Ephy[i]
    else:
        EntrP[i] = 0
        EntrN[i] = 0         
        Pepi[i] = Pepi[i-1] + IepiP[i] + EntrP[i]  - SedP[i] - Ep[i]#- PcPhy[i]
        Phyp[i] = Phyp[i-1] + Rp[i]    - EntrP[i]
        Psed[i] = Psed[i-1] + SedP[i]  - Rp[i]     + IsedP[i]   - Bp[i]

        Nepi[i] = Nepi[i-1] + IepiN[i] + EntrN[i]  - SedN[i] - En[i] - DenitN[i]- NcPhy[i] 
        Nhyp[i] = Nhyp[i-1] + Rn[i]    - EntrN[i]
        Nsed[i] = Nsed[i-1] + SedN[i]  - Rn[i]     + IsedN[i]   - Bn[i] 
        #Phy[i]  = Gr[i]     - MR[i]    - Sr[i]     - G[i]    - Ephy[i]


In [ ]:
    if DF['St'][i] <0:
        st = 0
    else:
        st = DF['St'][i]

    if (DF.Strat[i-1] == False and DF.Strat[i] == False): # Winter
        if DF.Season[i] == 'winter':
            SC = 1-(0.2*st/15)  
        else:
            SC = 0.1
    else:
        SC = 0.1#1-(0.9*st/15)   